In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import math
from concurrent.futures import ThreadPoolExecutor

%matplotlib inline

In [4]:
with open('processed\BadTracksHackaton1801', 'rb') as f:
    a = np.load(f)
a.shape

(1081, 7)